## Obserwator (ang. Observer, Publish-Subscribe, Event-Subscriber)

**Typ**: behawioralny \
**Zakres**: obiektowy

<div style="border: solid 1px;padding: 20px;text-align: center">
    Wzorzec <b>obserwator</b> definiuje zależność jeden-do-wielu między obiektami tak, że zmiana stanu jednego obiektu powoduje automatyczne powiadomienie i aktualizację wszystkich obiektów zależnych.
</div>

### Problem - powiadamianie wielu obiektów o zmianie

Kanał YouTube publikuje nowy film. Wszyscy subskrybenci powinni dostać **powiadomienie**.

**Problem:**
- Jak powiadomić wszystkich subskrybentów?
- Jak uniknąć sztywnego powiązania kanału z subskrybentami?
- Jak umożliwić subskrybowanie/odsubskrybowanie?

### Naiwne podejście - sztywne powiązanie

In [ ]:
class User:
    def __init__(self, name):
        self.name = name
    
    def notify(self, video):
        print(f"👤 {self.name}: Nowy film - {video}")


class YouTubeChannel:
    def __init__(self, name):
        self.name = name
        # ❌ Sztywne powiązanie - kanał zna konkretnych użytkowników
        self.user1 = None
        self.user2 = None
        self.user3 = None
    
    def upload_video(self, video):
        print(f"📺 {self.name}: Publikuję '{video}'\n")
        # ❌ Ręczne powiadamianie każdego
        if self.user1:
            self.user1.notify(video)
        if self.user2:
            self.user2.notify(video)
        if self.user3:
            self.user3.notify(video)

In [ ]:
channel = YouTubeChannel("Python Tutorials")

alice = User("Alice")
bob = User("Bob")

# "Subskrypcja" - ręczne przypisanie
channel.user1 = alice
channel.user2 = bob

channel.upload_video("Design Patterns")

**Problemy:**
- ❌ **Sztywne powiązanie** - kanał zna konkretnych użytkowników (`user1`, `user2`)
- ❌ **Ograniczona liczba** - co jeśli 100 subskrybentów? `user1...user100`?
- ❌ **Brak dynamiczności** - jak odsubskrybować?
- ❌ Kanał musi **znać** wszystkich subskrybentów

### Rozwiązanie - wzorzec Obserwator

**Idea:** Kanał (Subject) przechowuje **listę obserwatorów**. Powiadamia wszystkich na liście.

### Krok 1: Interfejs Observer

In [ ]:
from abc import ABC, abstractmethod

class Observer(ABC):
    """Interfejs dla obserwatorów"""
    
    @abstractmethod
    def update(self, message):
        """Otrzymuje powiadomienie"""
        pass

### Krok 2: Subject (Observable) - zarządza obserwatorami

In [ ]:
class Subject:
    """Subject - zarządza obserwatorami i powiadamia ich"""
    
    def __init__(self):
        self._observers = []  # Lista obserwatorów
    
    def attach(self, observer: Observer):
        """Dodaj obserwatora (subskrybuj)"""
        if observer not in self._observers:
            self._observers.append(observer)
            print(f"➕ Subskrybent dołączył")
    
    def detach(self, observer: Observer):
        """Usuń obserwatora (odsubskrybuj)"""
        if observer in self._observers:
            self._observers.remove(observer)
            print(f"➖ Subskrybent odszedł")
    
    def notify(self, message):
        """Powiadom wszystkich obserwatorów"""
        for observer in self._observers:
            observer.update(message)

**Kluczowy mechanizm:**
- `attach()` - dodaj obserwatora do listy
- `detach()` - usuń obserwatora z listy
- `notify()` - powiadom wszystkich na liście (pętla `for`)

### Krok 3: ConcreteSubject - kanał YouTube

In [ ]:
class YouTubeChannel(Subject):
    """Konkretny Subject - kanał YouTube"""
    
    def __init__(self, name):
        super().__init__()
        self.name = name
    
    def upload_video(self, video):
        print(f"\n📺 {self.name}: Publikuję '{video}'")
        # Powiadom wszystkich subskrybentów
        self.notify(f"Nowy film: {video}")

### Krok 4: ConcreteObserver - użytkownik

In [ ]:
class User(Observer):
    """Konkretny Observer - użytkownik"""
    
    def __init__(self, name):
        self.name = name
    
    def update(self, message):
        """Otrzymuje powiadomienie"""
        print(f"👤 {self.name}: {message}")

### Krok 5: Użycie - dynamiczne subskrybowanie

In [ ]:
# Tworzenie
channel = YouTubeChannel("Python Tutorials")
alice = User("Alice")
bob = User("Bob")
charlie = User("Charlie")

# Subskrybowanie
channel.attach(alice)
channel.attach(bob)

# Publikacja - powiadamia Alice i Bob
channel.upload_video("Design Patterns")

# Charlie dołącza
print()
channel.attach(charlie)

# Publikacja - powiadamia Alice, Bob i Charlie
channel.upload_video("SOLID Principles")

# Bob odsubskrybuje
print()
channel.detach(bob)

# Publikacja - powiadamia tylko Alice i Charlie
channel.upload_video("Clean Code")

**Zalety:**
- ✅ **Luźne powiązanie** - kanał nie zna konkretnych użytkowników
- ✅ **Dynamiczność** - attach/detach w dowolnym momencie
- ✅ **Skalowalność** - nieograniczona liczba obserwatorów
- ✅ **Open/Closed** - nowi obserwatorzy bez zmiany Subject

## Jak to działa? - wizualizacja

```python
channel.attach(alice)  # _observers = [alice]
channel.attach(bob)    # _observers = [alice, bob]

channel.upload_video("Design Patterns")
    ↓
    self.notify("Nowy film: Design Patterns")
        ↓
        for observer in [alice, bob]:
            observer.update("Nowy film: Design Patterns")
                ↓
                alice.update(...)  # "👤 Alice: Nowy film..."
                bob.update(...)    # "👤 Bob: Nowy film..."
```

**Subject powiadamia wszystkich obserwatorów na liście.**

## Struktura wzorca

**Elementy wzorca Obserwator:**

1. **Subject (Observable)** - `Subject`
   - Przechowuje listę obserwatorów (`_observers`)
   - Zarządza subskrypcją (`attach()`, `detach()`)
   - Powiadamia obserwatorów (`notify()`)

2. **ConcreteSubject** - `YouTubeChannel`
   - Konkretny subject
   - Przechowuje stan (np. nowy film)
   - Wywołuje `notify()` gdy stan się zmienia

3. **Observer** - `Observer`
   - Interfejs dla obserwatorów
   - Metoda `update()` - otrzymuje powiadomienie

4. **ConcreteObserver** - `User`
   - Konkretny obserwator
   - Implementuje `update()` - reaguje na powiadomienie

**Kluczowa właściwość:**
> Subject nie zna konkretnych obserwatorów - zna tylko interfejs `Observer`

## Przykład 2 - Stacja pogodowa

In [ ]:
from abc import ABC, abstractmethod

# ════════════════════════════════════════════════════════════
# Observer
# ════════════════════════════════════════════════════════════
class WeatherObserver(ABC):
    @abstractmethod
    def update(self, temperature: float, humidity: float):
        pass


# ════════════════════════════════════════════════════════════
# Subject - Stacja pogodowa
# ════════════════════════════════════════════════════════════
class WeatherStation:
    def __init__(self):
        self._observers = []
        self._temperature = 0
        self._humidity = 0
    
    def attach(self, observer: WeatherObserver):
        self._observers.append(observer)
    
    def detach(self, observer: WeatherObserver):
        self._observers.remove(observer)
    
    def notify(self):
        for observer in self._observers:
            observer.update(self._temperature, self._humidity)
    
    def set_measurements(self, temperature: float, humidity: float):
        """Nowe pomiary - powiadom obserwatorów"""
        print(f"\n🌡️  Nowe pomiary: {temperature}°C, wilgotność {humidity}%")
        self._temperature = temperature
        self._humidity = humidity
        self.notify()  # Automatyczne powiadomienie


# ════════════════════════════════════════════════════════════
# ConcreteObservers - Wyświetlacze
# ════════════════════════════════════════════════════════════
class CurrentConditionsDisplay(WeatherObserver):
    """Wyświetla aktualne warunki"""
    
    def update(self, temperature: float, humidity: float):
        print(f"📊 Aktualne warunki: {temperature}°C, wilgotność {humidity}%")


class StatisticsDisplay(WeatherObserver):
    """Wyświetla statystyki"""
    
    def __init__(self):
        self.temperatures = []
    
    def update(self, temperature: float, humidity: float):
        self.temperatures.append(temperature)
        avg = sum(self.temperatures) / len(self.temperatures)
        print(f"📈 Średnia temperatura: {avg:.1f}°C")


class ForecastDisplay(WeatherObserver):
    """Wyświetla prognozę"""
    
    def update(self, temperature: float, humidity: float):
        if temperature > 25:
            print(f"☀️  Prognoza: Słonecznie!")
        elif temperature > 15:
            print(f"⛅ Prognoza: Umiarkowanie")
        else:
            print(f"🌧️  Prognoza: Deszczowo")

In [ ]:
# Tworzenie
station = WeatherStation()

current = CurrentConditionsDisplay()
stats = StatisticsDisplay()
forecast = ForecastDisplay()

# Subskrybowanie
station.attach(current)
station.attach(stats)
station.attach(forecast)

# Pomiary - powiadamia wszystkie wyświetlacze
station.set_measurements(28, 65)
station.set_measurements(22, 70)
station.set_measurements(12, 80)

**Każdy wyświetlacz reaguje inaczej:**
- `CurrentConditionsDisplay` - pokazuje aktualne
- `StatisticsDisplay` - liczy średnią
- `ForecastDisplay` - przewiduje pogodę

## Warianty wzorca

**Wariant 1: Push model (domyślny)**

Subject **wysyła** dane do obserwatorów:
```python
def notify(self):
    for observer in self._observers:
        observer.update(self._temperature, self._humidity)  # Wysyła dane
```

**Wariant 2: Pull model**

Subject powiadamia, obserwator **pobiera** dane:
```python
def notify(self):
    for observer in self._observers:
        observer.update(self)  # Wysyła siebie (referencję)

# Observer
def update(self, subject):
    temp = subject.get_temperature()  # Pobiera dane sam
```

**Wariant 3: Event-based (z typem zdarzenia)**

```python
def notify(self, event_type, data):
    for observer in self._observers:
        observer.update(event_type, data)  # "video_uploaded", "comment_added"
```

## Kiedy używać wzorca Obserwator?

Wzorzec Obserwator stosuj gdy:

1. **Zmiana w jednym obiekcie wymaga zmiany w innych**
   - Liczba obiektów do powiadomienia nieznana z góry

2. **Chcesz uniknąć sztywnego powiązania**
   - Subject nie powinien znać konkretnych obserwatorów

3. **Dynamiczne subskrybowanie/odsubskrybowanie**
   - W runtime dodawanie/usuwanie obserwatorów

4. **Zdarzenie może mieć wielu odbiorców**
   - Event system

**Przykłady praktyczne:**
- GUI event handling (button click → handlers)
- Model-View-Controller (model → views)
- Publish-Subscribe systems (pub/sub)
- Event-driven architecture
- Notyfikacje (push notifications)
- Data binding (Angular, React)
- Reactive programming (RxJS, RxPy)

## Obserwator w praktyce - Python

Python ma wbudowane mechanizmy zbliżone do Obserwatora:

**1. Property z setterem:**
```python
class Data:
    def __init__(self):
        self._value = 0
        self.observers = []
    
    @property
    def value(self):
        return self._value
    
    @value.setter
    def value(self, val):
        self._value = val
        self._notify()  # Powiadom przy zmianie
    
    def _notify(self):
        for observer in self.observers:
            observer(self._value)
```

**2. Biblioteki:**
- `blinker` - signals and events
- `PyPubSub` - publish-subscribe
- `RxPy` - reactive extensions

## Podsumowanie

Wzorzec Obserwator:
- ✅ **Luźne powiązanie** - Subject nie zna konkretnych obserwatorów
- ✅ **Dynamiczność** - attach/detach w runtime
- ✅ **Skalowalność** - nieograniczona liczba obserwatorów
- ✅ **Open/Closed** - nowi obserwatorzy bez zmiany Subject
- ⚠️ **Kolejność** - niezdefiniowana kolejność powiadomień
- ⚠️ **Nieoczekiwane update** - obserwator może nie wiedzieć skąd update
- ⚠️ **Memory leaks** - zapomnienie detach może powodować wycieki

**Kluczowa idea:**
> **Jeden-do-wielu** - zmiana w Subject → powiadomienie wszystkich obserwatorów

**Struktura:**
```python
# Observer
class Observer(ABC):
    @abstractmethod
    def update(self, data):
        pass

# Subject
class Subject:
    def __init__(self):
        self._observers = []
    
    def attach(self, observer):
        self._observers.append(observer)
    
    def detach(self, observer):
        self._observers.remove(observer)
    
    def notify(self, data):
        for observer in self._observers:
            observer.update(data)

# ConcreteSubject
class ConcreteSubject(Subject):
    def change_state(self, new_state):
        self._state = new_state
        self.notify(new_state)  # Powiadom o zmianie
```

**Istota wzorca:**
- **Lista obserwatorów:** Subject przechowuje `_observers = []`
- **Powiadamianie:** Pętla `for observer in observers: observer.update()`
- **Luźne powiązanie:** Subject zna tylko interfejs `Observer`
- **Automatyzm:** Zmiana stanu → automatyczne powiadomienie

**Przepływ:**
1. Obserwator subskrybuje (`subject.attach(observer)`)
2. Stan Subject się zmienia
3. Subject wywołuje `notify()`
4. Wszyscy obserwatorzy otrzymują `update()`

**Alternatywne nazwy:**
- **Publish-Subscribe** (pub/sub)
- **Event-Subscriber**
- **Listener pattern**

## Idiomatyczne rozwiązanie w Pythonie

Wzorzec Obserwator używa **klas Observer** z metodą `update()`. Python oferuje prostsze rozwiązanie przez **funkcje jako callbacki** (bez klas Observer).

### Dwa problemy rozwiązywane przez Obserwator:

**Problem 1: Powiadamianie bez stanu**
```python
# Wystarczy wywołać funkcję przy zmianie
def notify_user(message):
    print(f"Powiadomienie: {message}")
```

**Problem 2: Obserwator ze stanem**
```python
# StatisticsDisplay musi pamiętać poprzednie temperatury
class StatisticsDisplay:
    def __init__(self):
        self.temperatures = []  # Stan!
```

### Pythoniczne rozwiązania

**Dla Problemu 1** (bez stanu) → funkcje jako callbacki ✅  
**Dla Problemu 2** (ze stanem) → klasy są potrzebne ⚠️

Zobaczmy obie sytuacje:

### Klasyczny wzorzec - klasy Observer

In [ ]:
from abc import ABC, abstractmethod

# ❌ Wzorzec: Interfejs Observer + klasy ConcreteObserver
class Observer(ABC):
    @abstractmethod
    def update(self, message):
        pass

class Subject:
    def __init__(self):
        self._observers = []
    
    def attach(self, observer: Observer):
        self._observers.append(observer)
    
    def notify(self, message):
        for observer in self._observers:
            observer.update(message)

class User(Observer):
    def __init__(self, name):
        self.name = name
    
    def update(self, message):
        print(f"👤 {self.name}: {message}")

class YouTubeChannel(Subject):
    def upload_video(self, video):
        print(f"\n📺 Nowy film: {video}")
        self.notify(f"Nowy film - {video}")


# Użycie
print("=== Wzorzec (klasy Observer) ===")
channel = YouTubeChannel()
alice = User("Alice")
bob = User("Bob")

channel.attach(alice)
channel.attach(bob)
channel.upload_video("Design Patterns")

### ✅ Idiom Pythona - funkcje jako callbacki

In [ ]:
# ✅ IDIOM: Lista callbacków (funkcji)
class YouTubeChannel:
    def __init__(self):
        self._callbacks = []  # Lista funkcji, nie obiektów Observer!
    
    def subscribe(self, callback_func):
        """Dodaj funkcję callback"""
        self._callbacks.append(callback_func)
    
    def unsubscribe(self, callback_func):
        """Usuń funkcję callback"""
        self._callbacks.remove(callback_func)
    
    def upload_video(self, video):
        print(f"\n📺 Nowy film: {video}")
        # Wywołaj wszystkie callbacki
        for callback in self._callbacks:
            callback(f"Nowy film - {video}")


# Callbacki jako funkcje/lambdy (bez klas Observer!)
def alice_notify(message):
    print(f"👤 Alice: {message}")

def bob_notify(message):
    print(f"👤 Bob: {message}")

# Lub lambda z zamknięciem
def create_user_callback(name):
    def callback(message):
        print(f"👤 {name}: {message}")
    return callback


# Użycie
print("\n=== Idiom Pythona (callbacki) ===")
channel = YouTubeChannel()

# Subskrybuj przekazując funkcje
channel.subscribe(alice_notify)
channel.subscribe(bob_notify)
channel.subscribe(create_user_callback("Charlie"))

channel.upload_video("SOLID Principles")

**Co się zmieniło?**
- ❌ Nie ma interfejsu `Observer` ani klas `ConcreteObserver`
- ❌ Nie ma metody `update()` - tylko wywołanie funkcji
- ✅ Lista **funkcji** (`_callbacks`), nie obiektów
- ✅ Obserwatorzy = zwykłe funkcje (lub lambdy, lub zamknięcia)
- ✅ **Prostsze** - funkcje zamiast klas

**Dlaczego to działa?**
- Funkcje w Pythonie są obiektami pierwszej klasy
- Możesz przechowywać funkcje w liście
- Wywołanie: `callback(message)` zamiast `observer.update(message)`
- Brak potrzeby definiowania klas jeśli nie ma stanu

### Kiedy klasy są wciąż potrzebne? ⚠️

In [ ]:
# Gdy obserwator potrzebuje stanu
class WeatherStation:
    def __init__(self):
        self._callbacks = []
        self._temperature = 0
    
    def subscribe(self, callback):
        self._callbacks.append(callback)
    
    def set_temperature(self, temp):
        self._temperature = temp
        print(f"\n🌡️  Nowa temperatura: {temp}°C")
        for callback in self._callbacks:
            callback(temp)


# ⚠️ KLASA POTRZEBNA - StatisticsDisplay musi pamiętać historię
class StatisticsDisplay:
    """Wyświetla statystyki - musi pamiętać poprzednie wartości"""
    
    def __init__(self):
        self.temperatures = []  # Stan - historia temperatur
    
    def update(self, temperature):
        self.temperatures.append(temperature)
        avg = sum(self.temperatures) / len(self.temperatures)
        print(f"📈 Średnia temperatura: {avg:.1f}°C")


# Proste callbacki - bez stanu
def current_display(temp):
    print(f"📊 Aktualna: {temp}°C")

def forecast_display(temp):
    if temp > 25:
        print(f"☀️  Prognoza: Słonecznie!")
    else:
        print(f"🌧️  Prognoza: Deszczowo")


# Użycie
station = WeatherStation()

# Proste callbacki - funkcje
station.subscribe(current_display)
station.subscribe(forecast_display)

# Obserwator ze stanem - metoda klasy (bound method)
stats = StatisticsDisplay()
station.subscribe(stats.update)  # Przekazujemy metodę (z self!)

station.set_temperature(28)
station.set_temperature(22)
station.set_temperature(18)

**W tym przykładzie:**
- `current_display`, `forecast_display` - proste funkcje (bez stanu) ✅
- `StatisticsDisplay` - klasa ze stanem (`self.temperatures`) ⚠️
- Python pozwala mieszać: `station.subscribe(stats.update)` - metoda jako callback!

**Kluczowa obserwacja:**
- Przekazujemy **metodę** `stats.update` (bound method)
- Metoda "pamięta" `self` - ma dostęp do `self.temperatures`
- Subject nie wie że to metoda - dla niego to zwykły callable

## Porównanie: Funkcje vs Klasy

| Aspekt | Funkcje jako callbacki | Klasy Observer |
|--------|------------------------|----------------|
| **Powiadamianie** | ✅ `callback(data)` | ✅ `observer.update(data)` |
| **Lista obserwatorów** | ✅ Lista funkcji | ✅ Lista obiektów |
| **Stan obserwatora** | ❌ Trudne (closure/globale) | ✅ `self.attribute` |
| **Prostota** | 🎯 Bardzo prosty kod | ⚠️ Interfejsy + klasy |
| **Elastyczność** | ✅ Funkcje, lambdy, metody | ⚠️ Tylko obiekty Observer |
| **Kiedy używać** | Proste notyfikacje | Obserwatorzy ze stanem |

## Kiedy używać funkcji, a kiedy klas?

### ✅ Użyj funkcji jako callbacki gdy:
- **Proste powiadamianie** - obserwator reaguje bez stanu
- **Event handlers** - GUI, web frameworks
- **Brak hierarchii** - nie potrzebujesz interfejsu Observer
- **Elastyczność** - różne typy callbacków (funkcje, lambdy, metody)

**Przykłady:** Button callbacks, event handlers, simple notifications

### ⚠️ Użyj klas Observer gdy:
- **Obserwator ze stanem** - musi pamiętać dane między powiadomieniami
- **Złożona logika** - obserwator robi więcej niż prosta reakcja
- **Interfejs wymagany** - potrzebujesz jasnego kontraktu (np. framework)
- **Wielokrotne metody** - obserwator ma `update()`, `reset()`, itp.

**Przykłady:** MVC views (przechowują stan), statystyki, agregacje danych

### 🎯 Najlepsze rozwiązanie: Mieszaj!
```python
station.subscribe(simple_function)      # Funkcja - prosta reakcja
station.subscribe(stats.update)         # Metoda - ze stanem (self)
station.subscribe(lambda t: print(t))   # Lambda - inline callback
```

Python pozwala przekazywać **dowolne callable** (funkcje, metody, lambdy, obiekty z `__call__`)!

## Podsumowanie

**Pythoniczny idiom:**
- Dla **prostych powiadomień** → funkcje jako callbacki ✅
- Dla **obserwatorów ze stanem** → klasy (ale przekaż metodę!) ⚠️
- **Mieszaj** - lista callbacków akceptuje wszystko co jest callable 🎯

**Kluczowa różnica:**
```python
# Idiom Pythona - callbacki
channel.subscribe(lambda msg: print(msg))  # ✅ Wystarczy!

# Wzorzec - klasy ze stanem
class Stats(Observer):  # ⚠️ Potrzebna gdy masz self.history
    def __init__(self):
        self.history = []
    
    def update(self, data):
        self.history.append(data)
```

**Zastosowanie idiomu:**
- Python preferuje **callables** nad interfejsy
- Funkcje pierwszej klasy eliminują potrzebę klas Observer
- Ale stan wciąż wymaga obiektów (można przekazać metodę!)
- Framework nie musi wiedzieć czy to funkcja czy metoda - **duck typing**!